In [1]:
import os

In [2]:
%pwd

'd:\\MAIN_PROJECTS\\DataScience_Project\\research'

In [3]:
os.chdir("../")
%pwd

'd:\\MAIN_PROJECTS\\DataScience_Project'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainingConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [6]:
from src.DataScience.constants import *
from src.DataScience.utils.common import read_yaml,create_directories

In [15]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
    def get_model_trainier_config(self) -> ModelTrainingConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainingConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        )
        return model_trainer_config

In [16]:
import pandas as pd
import os
from src.DataScience import logger
from sklearn.linear_model import ElasticNet
import joblib

In [17]:
class ModelTrainer:
    def __init__(self,config:ModelTrainingConfig):
        self.config = config
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        train_x = train_data.drop([self.config.target_column],axis=1)
        test_x = test_data.drop([self.config.target_column],axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]
        
        lr = ElasticNet(alpha=self.config.alpha,l1_ratio=self.config.l1_ratio)
        lr.fit(train_x,train_y)
        
        joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_name))
        

In [18]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainier_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-06-23 11:09:13,434: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-23 11:09:13,436: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-23 11:09:13,437: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-23 11:09:13,439: INFO: common: Created directory at: artifacts]
[2025-06-23 11:09:13,440: INFO: common: Created directory at: artifacts/model_trainer]
